In [8]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError,ChunkedEncodingError
import itertools
import multiprocessing
import Ipynb_importer
from IPpool import GetHeaders
from IPpool import Ip_pool
import random
import copy
# import requests_toolbelt.adapters.appengine

# requests_toolbelt.adapters.appengine.monkeypatch()


importing Jupyter notebook from IPpool.ipynb


In [9]:
#**************将装机地址变成一组一组*****************
def get_chunk(adr_ls,n):
    new_adr_ls = copy.deepcopy(adr_ls)
    adr_chunk = [new_adr_ls[i:i + n] for i in range(0, len(new_adr_ls), n)]
    return adr_chunk

In [10]:

#设置谷歌浏览器的参数
def set_chrome_param():
    #PROXY = '175.155.24.61:808'
    options = webdriver.ChromeOptions()
    chrome_path = '/home/zhh/下载/chromedriver'  
    #options.add_argument('--proxy-server={0}'.format(PROXY))
    # 设置成中文
    options.add_argument('lang=zh_CN.UTF-8')
    # 添加头部
    #1允许所有图片；2阻止所有图片；3阻止第三方服务器图片
    prefs = {'profile.default_content_setting_values': { 'images':2}}
    options.add_experimental_option('prefs', prefs)
    options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"')
    chrome = webdriver.Chrome(chrome_path,chrome_options=options)
    return chrome


In [11]:
#requests请求网页，返回一个BeautifulSoup对象
def requests_visit_url(url,prox_ip=None):
    headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":"http://www.dianping.com/shop/92374637",
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}
    headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
    #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"}
    is_continue = True
    num = 0
    while is_continue:
        num +=1
        print("访问{0}次".format(num))
        try:
            if prox_ip is None:
                res = requests.get(url,headers = headers)  #访问url，不设代理ip访问
            else:
                http = prox_ip[0]
                ip = http+"://"+prox_ip[1]
                proxies={http:ip}
                res = requests.get(url,headers = headers,proxies = proxies) #设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            is_continue = False 
            
        except HTTPError as e:
            print("断网")
            time.sleep(2)
            
        except Timeout as e:
            print("请求超时")
            time.sleep(2)
            
        except ConnectionError as e:
            print("访问被拒")
            is_continue = True
            time.sleep(2)
    return soup

In [12]:
#缓存结果
def save_cache(result,filename):
    with open(filename,'wb') as f1:
        pickle.dump(result,f1)

In [13]:
#加载缓存
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

In [14]:
# 切换路径
def get_spc_path(father_path,child_path):
    os.chdir(father_path)
    if os.path.exists(child_path):
        os.chdir(child_path)
    else:
        os.mkdir(child_path)
        os.chdir(child_path)
    print("done!!")

## 1.二级分类层

In [15]:
%%time
#获取所有的二级分类url
def classify(city_url,city,big_class):
    soup = requests_visit_url(city_url)
    clf_ls = []
    for i in soup.select("#classfy a"):
        clf_url = i["href"] #分类网址
        clf_tl = i.text.strip() # 分类标题
        clf_ls.append([big_class,city,clf_tl,clf_url])
    return clf_ls

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs


In [16]:
# %%time
# clf_ls = classify("http://www.dianping.com/search/category/7/10",'深圳',"美食")

## 2.区县层

In [17]:
def get_region(class_ls=None,prox_ip=None):
    print('正在爬取{0}'.format(class_ls[-2]))
    url = class_ls[-1]  #url
    soup = requests_visit_url(url,prox_ip) #访问
    region_ls = [] #地区的列表
    for i in soup.select("#region-nav a"):
        if i.text.strip() != "不限":
            region_url = i["href"] 
            region_tl = i.text.strip()
            region = class_ls[:-1]
            region.extend([region_tl,region_url])
            region_ls.append(region)
    return region_ls


In [18]:
# ip = Ip_pool(['http://www.xicidaili.com/nn/'])
# ip_pool = ip.get_ip_pool(test_url='http://www.dianping.com/search/category/7/10/g104',timeout =1)

In [19]:
# %%time
# prox_ip = random.choice(ip_pool)
# region_ls = []
# for i in clf_ls:
#     print(i)
#     sub_region = get_region(i,prox_ip)
#     region_ls.append(sub_region)

In [20]:
# region_ls

## 3.商圈层

In [21]:
def get_circle(region_ls=None):
    url = region_ls[-1]   #url网址
    soup = requests_visit_url(url) #访问BeautifulSoup
    circle_ls =[]             
    for i in soup.select("#region-nav-sub a"):  #商圈信息
        if i.span.text.strip()!="不限":   #剔除掉不限的url
            circle_url = i["href"]       #商圈url
            circle_tit = i.span.text.strip()  #商圈名
            circle = region_ls[:-1] 
            circle.extend([circle_tit,circle_url]) 
            circle_ls.append(circle)
            
    #如果没有商圈的话，就拿原来的url当作是商圈的url
    if circle_ls==[]:
        print(region_ls[-2]+"没有子分类！！")
        region_ls.insert(4,"没有商圈")
        circle_ls.append(region_ls)

    return circle_ls
    

In [22]:
# %%time
# circle_ls = get_circle(['美食',
#   '广州',
#   '茶餐厅',
#   '天河区',
#   'http://www.dianping.com/search/category/4/10/g207r22'])

## 4.商家信息层

In [23]:
#获取每个商家的链接、店名
def get_stores(circle_ls = None):
    store_ls=[] #存储的列表
    url = circle_ls[-1]  #url
    soup = requests_visit_url_test(url,is_use_ip=True,timeout=2)  #访问当前主页
    is_continue = True
    page = 0
    while is_continue:
        page += 1
        print("正在爬取第{0}页".format(page))
        
        #解析数据
        if soup.select(".content") ==[]:
            print("此页没有任何信息！！")
            store_ls = []
            is_continue = False 
            
        else:
            for i in soup.select(".content"):  
                for j in i.select(".pic"):
                    store_name = j.img["alt"]   #店名
                    link = j.a["href"]     #详情层
                    store = circle_ls[:5]  #只取前5位
                    store.extend([url,store_name,link]) 
                    store_ls.append(store)


            #获取下一页的数据
            time.sleep(2)

            next_tag = soup.find_all("a",text = re.compile("下一页"))  #获取字符属性为下一页的url
            if next_tag != []:                                         #保证含有字符属性
                url = next_tag[0]["href"]                              #获取下一页的url
                soup = requests_visit_url(url)                         #更新BeautifulSoup对象
                time.sleep(2)
            else:
                url = ''
                is_continue  = False                                  #如果没有下一页，就停止更新
    
    return store_ls

In [24]:
# %%time
# store_ls = get_stores(['美食',
#   '广州',
#   '茶餐厅',
#   '天河区',
#   '天河城/体育中心',
#   'http://www.dianping.com/search/category/4/10/g207r1519'])


###### get_info_vesion2()

## 5.详情层

In [25]:
#获取详情层的数据，如星级用户、浏览人数、均价
def get_info(store_ls=None):
    
    #首先先保证导入的数据不为空
    if store_ls !=[]:
        url = store_ls[-1] #详情信息表
        soup = requests_visit_url(url)
        try:
            start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
        except Exception as e:
            print("start",e)
            start = ''

        try:
            review_count = soup.select("#reviewCount")[0].text  #浏览人数
        except Exception as e:
            review_count = ''
            print("review_coun",e)

        try:
            avgprice = soup.select("#avgPriceTitle")[0].text   #均价
        except Exception as e:
            avgprice = ''
            print("avgprice",e)

        #评分系统
        try:
            score= []
            for j in soup.select("#comment_score .item"):
                score.append(j.text)
            score = "|".join(score)

        except Exception as e:
            score = ""
            print("score",e)

        try:   
            address = soup.select(".address .item")[0].text.strip() #地址
        except Exception as e:
            address = ''
            print("address出错了",e)

        #电话号码
        try:
            tel =[]
            for i in soup.select(".tel .item"):
                tel.append(i.text)
            tel = "|".join(tel)

        except:
            tel=""
            print("tel出错了",e)

        #路径
        try:
            path = []
            for i in soup.select(".breadcrumb a"):
                path.append(i.text.strip())
            path ='>'.join(path)
        except Exception as e:
            print("path出错了",e)
            path =''

        info = store_ls[:]
        info.extend([start,review_count,avgprice,score,address,tel,path])
    return info


In [26]:
# get_info(['美食',
#   '广州',
#   '茶餐厅',
#   '天河区',
#   '天河城/体育中心',
#   'http://www.dianping.com/search/category/4/10/g207r1519p2',
#   '粤港烧卤鸭饭(天河店)',
#   'http://www.dianping.com/shop/94311152'])

## 6 主程序

In [ ]:
%%time

clf_ls = classify("http://www.dianping.com/search/category/4/10",'广州',"美食")
print("所有二级分类已经获取完毕")

region_ls = list(map(get_region,clf_ls))
print("获取所有的区域清单") 

In [ ]:
#解链
region_ls = list(itertools.chain.from_iterable(region_ls))
region_ls

In [ ]:
%%time
#***************获取所有的商圈信息****************************888
circle_ls = list(map(get_circle,region_ls))

In [ ]:
#解链
circle_ls = list(itertools.chain.from_iterable(circle_ls))

In [27]:
def requests_visit_url_test(url,is_use_ip =False,timeout=3):
        global ip_pool
        global bad_ip
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":"http://www.dianping.com/shop/92374637",
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}

        #获取任意一个请求头
        headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        bad_ip = []  #用来装坏的ip
        
        #*********如果不设置ip的话**********************************
        if is_use_ip == False:
            res = requests.get(url,headers = headers,timeout=timeout)  #访问url，不设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            if soup.title.text == '403 Forbidden':
                print("ip被禁止了")
                raise ConnectionError
           
        #************如果设置ip的话，则执行这一段*********************
        else:
            is_continue = True   #设置循环标志
            while is_continue:
            #整理成ip地址的格式
                try:
                    #
                    if len(ip_pool)==len(bad_ip):
                        ip_url = ['http://www.xicidaili.com/nn/']
                        test_url = "http://www.dianping.com/shop/2954893"
                        ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)
                        prox_ip = random.choice(ip_pool)
                        bad_ip = []
                        
                    #******随机生成一个ip*******
                    prox_ip = random.choice(ip_pool)
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    
                    #*******用代理ip访问********
                    res = requests.get(url,headers = headers,proxies = proxies,timeout=timeout) #设代理ip访问
                    html =  res.content.decode('utf-8')
                    soup  =  BeautifulSoup(html,'lxml')
            
                    #********如果返回网页被禁止的情况，触发异常,否则返回正常**********
                    if soup.title.text == '403 Forbidden':
                        print("ip被禁止了")
                        raise ConnectionError
                    else:
                        is_continue = False

                except HTTPError as e:
                    bad_ip.append(prox_ip)
                    print("断网")
                    prox_ip = random.choice(ip_pool)


                except Timeout as e:
                    bad_ip.append(prox_ip)
                    print("超时")
                    prox_ip = random.choice(ip_pool)


                except ConnectionError as e:
                    print("访问被拒")
                    prox_ip = random.choice(ip_pool)

        return soup

In [ ]:
save_cache(circle_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_circle_ls.pickle")

In [28]:
circle_ls = read_cache("/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_circle_ls.pickle")
print(len(circle_ls))
circle_chunk = get_chunk(circle_ls,5)
len(circle_chunk)

3298


660

In [30]:
offset=283
save_cache(offset,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_offset.pickle")

In [ ]:
%%time
if __name__=='__main__':
    
    
    is_continue = True
    while is_continue:

        try:
            global ip_pool
            ip_url = ['http://www.xicidaili.com/nn/']
            test_url = "http://www.dianping.com/shop/2954893"
            ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)

            offset = read_cache("/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_offset.pickle")
            end = 660
            
            #***********多进程处理*************************
            for index,chunk in enumerate(circle_chunk[offset:end]):
                offset_index = offset + index
                print("正在爬第{0}chunk".format(offset_index))
                pool = multiprocessing.Pool(processes=3) # 创建3个进程
                result = []
                for i in chunk:
                    result.append(pool.apply_async(get_stores, (i, )))
                pool.close() # 关闭进程池，表示不能再往进程池中添加进程，需要在join之前调用
                pool.join() # 等待进程池中的所有进程执行完毕
                
                #***********遍历每个进程的结果*****************
                store_ls = []
                for res in result:
                    store_ls.append(res.get())
                
                #***********获取每一个chunk的内容****************
                print("第{0}chunk已经爬完了".format(offset_index))
                store_ls = list(itertools.chain.from_iterable(store_ls))
                save_cache(store_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_store_ls_{0}_chunk.pickle".format(offset_index))
                
                #********爬完直接将偏移量设为下一个*************************
                last_offset_1 = offset_index+1
                save_cache(last_offset_1,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_offset.pickle")
                
        #***************如果出现ChunkedEncodingError重复执行这段代码*********************     
        except ChunkedEncodingError as e:
            print("出现ChunkedEncodingError")
            last_offset_2 = offset_index-1
            print("保存上次的offset:{0}".format(last_offset_2))
            save_cache(last_offset_2,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_offset.pickle")
            continue
            
        #***************如果出现ChunkedEncodingError重复执行这段代码*********************      
        except KeyboardInterrupt as e:
            print("手动停止")
            last_offset_3 = offset_index
            print（"保存现有的offset:{0}".format(last_offset_3)）
            save_cache(last_offset_3,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_offset.pickle")
            break
#     print("***********准备爬商家的详情信息******************")
#     pool = multiprocessing.Pool(processes=3) # 创建3个进程
#     result=[]
#     for i in store_ls:
#         result.append(pool.apply_async(get_info,(i,)))
#     pool.close()
#     pool.join()
    
#     print("信息页已经爬完了！！")
#     info_ls=[]
#     for res in result:
#         info_ls.append(res.get())
#     save_cache(info_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_info_test_ls.pickle")